In [36]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re

In [37]:
#read pu with both pu ID and geometry of each point
pu_geometry = gpd.read_file(r'C:\Users\kevan\OneDrive\Desktop\Data+\DPS-Planning\GIS_files\pu_gen.geojson')

In [38]:
#loading in the current enrollment for only the 2024-25 school year
current_enrollment = pd.read_csv(r'C:\Users\kevan\OneDrive\Desktop\Data+\marketshare_output_pu_2324_848_gr_2021_2425_long.csv')
current_enrollment_2024 = current_enrollment[current_enrollment['year_str']=='2024-2025']
current_enrollment_2024.head()
current_enrollment_2024 = current_enrollment_2024.rename(columns = {'pu_2324_848':'pu_2324_84'})

In [39]:
#now we want to only show the 5th, 6th, 7th, and 8th grade - the grades that will be in high school in 2028-29
mask = current_enrollment_2024['grade'].isin([5,6,7,8])
hs_2028 = current_enrollment_2024[mask]
hs_2028 = hs_2028[['pu_2324_84','count','basez','lottery']]

In [43]:
#merging identical rows to get student counts by block and grade
columns_to_group = ['pu_2324_84']
hs_2028 = hs_2028.groupby(columns_to_group,as_index=False).sum() #as_index=False preserves the columns we are grouping by

#the planning units with zero enrollment are missing, so we must fill them with zeros
all_pus = pd.DataFrame({'pu_2324_84': range(1,849)})
hs_2028_full = all_pus.merge(hs_2028, on='pu_2324_84',how='left').fillna(0)
hs_2028_full.head()

,pu_2324_84,count,basez,lottery
0,1,1.0,0.0,1.0
1,2,9.0,4.0,4.0
2,3,2.0,2.0,0.0
3,4,2.0,1.0,1.0
4,5,8.0,6.0,1.0


In [ ]:
# merge basez student count onto the planning unit pu_geometry file so that it contains both geometry and student count. merge by pu ID.
pu_geometry = pu_geometry.rename(columns={'OBJECTID': 'pu_2324_84'})
pu_geometry_with_curr = pu_geometry.merge(
    hs_2028_full[['pu_2324_84', 'basez']],
    on='pu_2324_84',
    how='left'  
)

pu_geometry_with_curr

,pu_2324_84,student_gen,geometry,basez
0,1,0,"MULTIPOLYGON (((-78.82256 36.19379, -78.82287 ...",0.0
1,2,0,"MULTIPOLYGON (((-78.86225 36.05177, -78.85945 ...",4.0
2,3,0,"MULTIPOLYGON (((-78.79371 35.94418, -78.79394 ...",2.0
3,4,0,"MULTIPOLYGON (((-78.98659 35.88678, -78.98626 ...",1.0
4,5,0,"MULTIPOLYGON (((-78.7536 36.03136, -78.74348 3...",6.0


In [47]:
pu_geometry_with_curr['final_proj'] = round(0.3*pu_geometry_with_curr['student_gen']) + pu_geometry_with_curr['basez']
pu_geometry_with_curr.head(30)

,pu_2324_84,student_gen,geometry,basez,final_proj
0,1,0,"MULTIPOLYGON (((-78.82256 36.19379, -78.82287 ...",0.0,0.0
1,2,0,"MULTIPOLYGON (((-78.86225 36.05177, -78.85945 ...",4.0,4.0
2,3,0,"MULTIPOLYGON (((-78.79371 35.94418, -78.79394 ...",2.0,2.0
3,4,0,"MULTIPOLYGON (((-78.98659 35.88678, -78.98626 ...",1.0,1.0
4,5,0,"MULTIPOLYGON (((-78.7536 36.03136, -78.74348 3...",6.0,6.0
5,6,115,"MULTIPOLYGON (((-78.81683 35.90901, -78.81684 ...",3.0,37.0
6,7,0,"MULTIPOLYGON (((-78.89573 35.88917, -78.89509 ...",9.0,9.0
7,8,0,"MULTIPOLYGON (((-78.89205 35.99003, -78.89203 ...",10.0,10.0
8,9,0,"MULTIPOLYGON (((-78.86251 35.98147, -78.86303 ...",28.0,28.0
9,10,0,"MULTIPOLYGON (((-78.89244 35.95776, -78.89243 ...",16.0,16.0


In [ ]:
#pu_geometry_with_curr.to_file('pu_with_proj.geojson', driver = 'geojson')